In [ ]:
# Finding insides from AirBnB in Berlin

In this project I used the [Cross Industry Process](https://en.wikipedia.org/wiki/Cross-industry_standard_process_for_data_mining) for Data Mining on the [Berlin AirBnB dataset](http://insideairbnb.com/get-the-data.html). I hope this project will give you some valuable insides for Berlin.

In [ ]:
## Business Understanding


In [ ]:
## Data Understanding

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from helperLibrary import *

ModuleNotFoundError: No module named 'sklearn'

In [3]:
Listings = pd.read_csv('./listings 2.csv')
Calendar = pd.read_csv('./calendar.csv')

/Users/ibeketov/dev/nano_ai/project1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (38,43,61,62,84,85,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Checking the shape of the datasets
Listings.shape
Calendar.shape

(24586, 106)

In [10]:
colinspection(Listings)

NameError: name 'colinspection' is not defined

In [ ]:
Listings_df.dtypes